In [25]:
#pip install ipykernel
#pip install pydantic
#!pip install email-validator

## Define and Use a Pydantic Model (v1 & v2 compatible)

In [4]:
from pydantic import BaseModel, Field
from typing import List, Optional

class User(BaseModel):
    id: int
    name: str
    email: str
    age: Optional[int] = Field(default=None, ge=0, le=120)
    interests: List[str] = []

# Creating a user instance
user_data = {
    "id": 1,
    "name": "Alice",
    "email": "alice@example.com",
    "age": 30,
    "interests": ["reading", "traveling"]
}

user = User(**user_data)

# Print user as an object
print(user)

# Correct way to print as JSON in Pydantic v2
print(user.model_dump_json(indent=2))

id=1 name='Alice' email='alice@example.com' age=30 interests=['reading', 'traveling']
{
  "id": 1,
  "name": "Alice",
  "email": "alice@example.com",
  "age": 30,
  "interests": [
    "reading",
    "traveling"
  ]
}


In [5]:
bad_user_data = {
    "id": "not-a-number",
    "name": "Bob",
    "email": "bob@example.com",
    "age": -5
}

try:
    user = User(**bad_user_data)
except Exception as e:
    print("Validation Error:", e)

Validation Error: 2 validation errors for User
id
  Input should be a valid integer, unable to parse string as an integer [type=int_parsing, input_value='not-a-number', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/int_parsing
age
  Input should be greater than or equal to 0 [type=greater_than_equal, input_value=-5, input_type=int]
    For further information visit https://errors.pydantic.dev/2.10/v/greater_than_equal


## Nested Models

In [6]:
from pydantic import BaseModel
from typing import List

class Address(BaseModel):
    street: str
    city: str
    zipcode: str

class User(BaseModel):
    id: int
    name: str
    email: str
    address: Address

user_data = {
    "id": 101,
    "name": "John Doe",
    "email": "john@example.com",
    "address": {
        "street": "123 Main St",
        "city": "Springfield",
        "zipcode": "12345"
    }
}

user = User(**user_data)
print(user.model_dump_json(indent=2))

{
  "id": 101,
  "name": "John Doe",
  "email": "john@example.com",
  "address": {
    "street": "123 Main St",
    "city": "Springfield",
    "zipcode": "12345"
  }
}


## Automatic Type Conversion

In [7]:
from pydantic import BaseModel

class Product(BaseModel):
    id: int
    price: float
    in_stock: bool

product = Product(id='10', price='199.99', in_stock='true')
print(product)

id=10 price=199.99 in_stock=True


## Validation with Custom Errors

In [10]:
from pydantic import BaseModel, validator

class Signup(BaseModel):
    username: str
    password: str

    @validator("password")
    def strong_password(cls, v):
        if len(v) < 8:
            raise ValueError("Password must be at least 8 characters long")
        return v

signup = Signup(username="user1", password="mysecurepwd")  # ✅
#signup = Signup(username="user1", password="123")  # ❌ Raises validation error
print(signup)

username='user1' password='mysecurepwd'


C:\Users\Abhishek Jaiswal\AppData\Local\Temp\ipykernel_5788\4112350707.py:7: PydanticDeprecatedSince20: Pydantic V1 style `@validator` validators are deprecated. You should migrate to Pydantic V2 style `@field_validator` validators, see the migration guide for more details. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  @validator("password")


## Environment-Based Settings with BaseSettings

In [11]:
from pydantic import BaseSettings

class Settings(BaseSettings):
    db_url: str
    redis_url: str = "redis://localhost"

    class Config:
        env_file = ".env"

settings = Settings()
print(settings.db_url)
print(settings.redis_url)

#.env file has DB_URL=postgresql://localhost/mydb

PydanticImportError: `BaseSettings` has been moved to the `pydantic-settings` package. See https://docs.pydantic.dev/2.10/migration/#basesettings-has-moved-to-pydantic-settings for more details.

For further information visit https://errors.pydantic.dev/2.10/u/import-error

## Model to Dictionary and Back

In [12]:
user_dict = user.model_dump()
print(user_dict)

# Create a model again from dict
user2 = User.model_validate(user_dict)
print(user2)

{'id': 101, 'name': 'John Doe', 'email': 'john@example.com', 'address': {'street': '123 Main St', 'city': 'Springfield', 'zipcode': '12345'}}
id=101 name='John Doe' email='john@example.com' address=Address(street='123 Main St', city='Springfield', zipcode='12345')


## Default Factories (e.g. UUIDs or timestamps)


In [13]:
from pydantic import BaseModel, Field
from uuid import uuid4
from datetime import datetime

class Request(BaseModel):
    id: str = Field(default_factory=lambda: str(uuid4()))
    timestamp: datetime = Field(default_factory=datetime.utcnow)

req = Request()
print(req.model_dump_json(indent=2))

{
  "id": "8d53151a-b09c-4753-a6db-e68fcc6d7949",
  "timestamp": "2025-04-13T15:09:49.744084"
}


##  Flexible Input: model_validate()

In [14]:
from pydantic import BaseModel

class Question(BaseModel):
    question: str
    answer: str

raw_input = {"question": "What is AI?", "answer": "Artificial Intelligence"}

q = Question.model_validate(raw_input)
print(q)

question='What is AI?' answer='Artificial Intelligence'


## Discriminated Unions (LLM tool or API payloads)

In [16]:
from pydantic import BaseModel, Field
from typing import Union, Literal

class TextInput(BaseModel):
    type: Literal["text"]
    content: str

class ImageInput(BaseModel):
    type: Literal["image"]
    url: str

InputData = Union[TextInput, ImageInput]

example = {"type": "image", "url": "http://example.com/image.png"}
data = InputData.model_validate(example)
print(data)
# Used in LLM tools like LangChain or Semantic Kernel when deciding input modality.

AttributeError: model_validate

## Model Composition / ConfigDict & computed_fields


In [17]:
from pydantic import BaseModel, computed_field

class Order(BaseModel):
    price_per_unit: float
    quantity: int

    @computed_field
    def total(self) -> float:
        return self.price_per_unit * self.quantity

order = Order(price_per_unit=20, quantity=3)
print(order.total)  # 60

60.0


## Custom Data Cleaning via model_post_init

In [18]:
from pydantic import BaseModel

class CleanedText(BaseModel):
    text: str

    def model_post_init(self, __context):
        self.text = self.text.strip().lower()

data = CleanedText(text="   HeLLo World  ")
print(data.text)  # 'hello world'

hello world


## Recursive / Tree Structures (useful in prompt tree parsing)

In [19]:
from typing import List, Optional
from pydantic import BaseModel

class Node(BaseModel):
    name: str
    children: Optional[List["Node"]] = None

Node.model_rebuild()

tree = Node(name="root", children=[
    Node(name="child1"),
    Node(name="child2", children=[
        Node(name="grandchild1")
    ])
])
print(tree.model_dump_json(indent=2))

{
  "name": "root",
  "children": [
    {
      "name": "child1",
      "children": null
    },
    {
      "name": "child2",
      "children": [
        {
          "name": "grandchild1",
          "children": null
        }
      ]
    }
  ]
}


## Parsing from JSON or YAML files

In [20]:
import json
from pydantic import BaseModel

class Config(BaseModel):
    model_name: str
    temperature: float
    top_p: float

with open("config.json") as f:
    cfg = Config.model_validate(json.load(f))

print(cfg)

FileNotFoundError: [Errno 2] No such file or directory: 'config.json'

## Serialization for APIs or LLM Logs

In [21]:
log = {
    "prompt": "What is machine learning?",
    "response": "Machine learning is a branch of AI..."
}

class LogEntry(BaseModel):
    prompt: str
    response: str

entry = LogEntry(**log)
with open("log.jsonl", "a") as f:
    f.write(entry.model_dump_json() + "\n")


##  Pydantic Example: Email Input Model

In [26]:
from pydantic import BaseModel, Field, EmailStr, computed_field
from typing import Optional, List
from datetime import datetime

# 🔹 Nested model for sender
class SenderInfo(BaseModel):
    name: Optional[str] = None
    email: EmailStr

# 🔹 Main model for Email input
class EmailData(BaseModel):
    id: str
    sender: SenderInfo
    subject: str
    body: str
    received_at: datetime = Field(default_factory=datetime.utcnow)
    tags: List[str] = Field(default_factory=list)
    priority: Optional[str] = Field(default=None, description="low | medium | high")

    # 🔹 Post init to sanitize subject/body (trim whitespace, normalize case)
    def model_post_init(self, __context):
        self.subject = self.subject.strip().title()
        self.body = self.body.strip()

        # Auto-set priority if body contains urgent keywords
        if not self.priority:
            if any(keyword in self.body.lower() for keyword in ['urgent', 'asap', 'immediately']):
                self.priority = "high"
            else:
                self.priority = "low"

    # 🔹 Computed field for short preview
    @computed_field
    def preview(self) -> str:
        return self.body[:60] + "..." if len(self.body) > 60 else self.body

    # 🔹 Add custom method to auto-tag
    def auto_tag(self):
        keywords = {
            "payment": "finance",
            "invoice": "finance",
            "meeting": "calendar",
            "support": "helpdesk",
            "error": "bug"
        }
        for word, tag in keywords.items():
            if word in self.body.lower() and tag not in self.tags:
                self.tags.append(tag)

# 🔸 Example usage
email_input = {
    "id": "email-001",
    "sender": {"name": "Alice", "email": "alice@example.com"},
    "subject": "   urgent issue with payment ",
    "body": "Hi, I noticed an error in my last invoice. Please fix it ASAP."
}

email = EmailData.model_validate(email_input)
email.auto_tag()  # simulate tagging after validation

# 🔸 Output result
print(email.model_dump_json(indent=2))

{
  "id": "email-001",
  "sender": {
    "name": "Alice",
    "email": "alice@example.com"
  },
  "subject": "Urgent Issue With Payment",
  "body": "Hi, I noticed an error in my last invoice. Please fix it ASAP.",
  "received_at": "2025-04-13T15:18:05.158593",
  "tags": [
    "finance",
    "bug"
  ],
  "priority": "high",
  "preview": "Hi, I noticed an error in my last invoice. Please fix it ASA..."
}


## Pydantic Model for ML Classification Input

In [27]:
from pydantic import BaseModel, Field, conint, confloat
from typing import Optional


class CustomerInput(BaseModel):
    age: conint(ge=18, le=100) = Field(..., description="Customer age in years", example=35)
    monthly_income: confloat(ge=0.0, le=100000.0) = Field(..., description="Monthly income in USD", example=4500.0)
    account_age_months: conint(ge=0, le=240) = Field(..., description="Number of months customer has had the account", example=36)
    contacted_support_last_30_days: bool = Field(False, description="Has the customer contacted support in last 30 days?")
    num_transactions_last_month: Optional[conint(ge=0, le=1000)] = Field(0, description="Number of transactions last month", example=25)
    satisfaction_score: Optional[confloat(ge=0.0, le=1.0)] = Field(0.7, description="Customer satisfaction score (0 to 1 scale)")

    class Config:
        schema_extra = {
            "example": {
                "age": 45,
                "monthly_income": 5500.0,
                "account_age_months": 48,
                "contacted_support_last_30_days": False,
                "num_transactions_last_month": 20,
                "satisfaction_score": 0.82
            }
        }

C:\Users\Abhishek Jaiswal\.conda\envs\myenv\Lib\site-packages\pydantic\_internal\_config.py:345: UserWarning: Valid config keys have changed in V2:
* 'schema_extra' has been renamed to 'json_schema_extra'
  warnings.warn(message, UserWarning)


In [ ]:
from fastapi import FastAPI
from model import CustomerInput  # assuming it's saved in model.py

app = FastAPI()

@app.post("/predict")
def predict(input: CustomerInput):
    # preprocess and pass to ML model
    return {"message": "Prediction done!", "input": input}